In [1]:
import pandas as pd 
import numpy as np 

In [ ]:
df = pd.read_csv("/Users/donjhaiholland/Downloads/U.S._Chronic_Disease_Indicators.csv")
#df.head()
#df.dtypes
#df.describe()

# Drop empty columns
nan_cols = df.columns[df.isna().all()]
nonan_cols = df.drop(columns=nan_cols)
nonan_cols.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,DataSource,Topic,Question,Response,DataValueUnit,DataValueType,...,TopicID,QuestionID,ResponseID,DataValueTypeID,StratificationCategoryID1,StratificationID1,StratificationCategoryID2,StratificationID2,StratificationCategoryID3,StratificationID3
0,2020,2020,US,United States,BRFSS,Health Status,Recent activity limitation among adults,NaN,Number,Age-adjusted Mean,...,HEA,HEA04,NaN,AGEADJMEAN,SEX,SEXF,NaN,NaN,NaN,NaN
1,2015,2019,AR,Arkansas,US Cancer DVT,Cancer,"Invasive cancer (all sites combined), incidence",NaN,Number,Number,...,CAN,CAN07,NaN,NMBR,SEX,SEXM,NaN,NaN,NaN,NaN
2,2015,2019,CA,California,US Cancer DVT,Cancer,"Cervical cancer mortality among all females, u...",NaN,Number,Number,...,CAN,CAN03,NaN,NMBR,OVERALL,OVR,NaN,NaN,NaN,NaN
3,2015,2019,CO,Colorado,US Cancer DVT,Cancer,"Invasive cancer (all sites combined), incidence",NaN,Number,Number,...,CAN,CAN07,NaN,NMBR,RACE,HIS,NaN,NaN,NaN,NaN
4,2015,2019,GA,Georgia,US Cancer DVT,Cancer,"Prostate cancer mortality among all males, und...",NaN,Number,Number,...,CAN,CAN05,NaN,NMBR,RACE,WHT,NaN,NaN,NaN,NaN


In [140]:
# Build diabetes table
diabetes = nonan_cols.loc[
    (nonan_cols["Topic"]=="Diabetes") & (nonan_cols["StratificationCategory1"]=="Overall") & 
    (nonan_cols["DataValueType"]=="Age-adjusted Prevalence")
]

# Select relevant columns
diab_y = diabetes[["LocationAbbr", "YearStart", "YearEnd",
    "DataValue", "LowConfidenceLimit", "HighConfidenceLimit",
    "Question", "QuestionID"
]].rename(columns={"DataValue":"diabetes_y", 
                   "LowConfidenceLimit":"diabetes_lcl",
                   "HighConfidenceLimit":"diabetes_ucl",
                   "Question":"diabetes_question",    
                   "QuestionID": "diabetes_question_id"})

# Ensure numeric
diab_y["diabetes_y"] = pd.to_numeric(diab_y["diabetes_y"], errors="coerce")

In [151]:
# Build poverty table
poverty = nonan_cols.loc[(nonan_cols['Topic']=="Social Determinants of Health") &
(nonan_cols["StratificationCategory1"]=="Overall") & (nonan_cols["QuestionID"]=="SDH01")]


pov_x = poverty[[
    "LocationAbbr", "YearStart", "YearEnd",
    "DataValue", "LowConfidenceLimit", "HighConfidenceLimit",
    "Question", "QuestionID"
]].rename(columns={"DataValue":"poverty_x", "LowConfidenceLimit":"poverty_lcl",
                   "HighConfidenceLimit":"poverty_ucl", "Question":"poverty_question",
                    "QuestionID":"poverty_question_id" })
pov_x

,LocationAbbr,YearStart,YearEnd,poverty_x,poverty_lcl,poverty_ucl,poverty_question,poverty_question_id
15515,AR,2019,2019,27.8,26.8,28.9,Living below 150% of the poverty threshold amo...,SDH01
18668,AK,2019,2019,17.7,15.8,19.9,Living below 150% of the poverty threshold amo...,SDH01
21571,AZ,2019,2019,21.9,21.4,22.6,Living below 150% of the poverty threshold amo...,SDH01
26877,AL,2019,2019,25.4,24.6,26.2,Living below 150% of the poverty threshold amo...,SDH01
32066,HI,2019,2019,14.6,13.3,16.0,Living below 150% of the poverty threshold amo...,SDH01
...,...,...,...,...,...,...,...,...
258688,TN,2021,2021,23.0,22.4,23.7,Living below 150% of the poverty threshold amo...,SDH01
259422,VT,2021,2021,16.5,14.6,18.6,Living below 150% of the poverty threshold amo...,SDH01
263677,WI,2021,2021,17.8,17.1,18.5,Living below 150% of the poverty threshold amo...,SDH01
267548,WY,2021,2021,19.2,17.3,21.3,Living below 150% of the poverty threshold amo...,SDH01


In [168]:
# Merge the two tables
analysis = pd.merge(diab_y, pov_x, on=["LocationAbbr", "YearStart", "YearEnd"], how='inner')

print("States:", analysis["LocationAbbr"].nunique())
analysis = analysis.dropna(subset=["diabetes_y", "poverty_x"]).reset_index(drop=True)

print(analysis.shape)
analysis.tail()

States: 52
(102, 13)


,LocationAbbr,YearStart,YearEnd,diabetes_y,diabetes_lcl,diabetes_ucl,diabetes_question,diabetes_question_id,poverty_x,poverty_lcl,poverty_ucl,poverty_question,poverty_question_id
97,VT,2021,2021,7.2,6.4,8.0,Diabetes among adults,DIA01,16.5,14.6,18.6,Living below 150% of the poverty threshold amo...,SDH01
98,WA,2021,2021,7.9,7.4,8.5,Diabetes among adults,DIA01,16.5,15.9,17.0,Living below 150% of the poverty threshold amo...,SDH01
99,WV,2021,2021,13.3,12.4,14.2,Diabetes among adults,DIA01,27.6,26.2,28.9,Living below 150% of the poverty threshold amo...,SDH01
100,WI,2021,2021,8.0,7.1,9.0,Diabetes among adults,DIA01,17.8,17.1,18.5,Living below 150% of the poverty threshold amo...,SDH01
101,WY,2021,2021,7.7,6.8,8.6,Diabetes among adults,DIA01,19.2,17.3,21.3,Living below 150% of the poverty threshold amo...,SDH01
